# Post-Process Ust_Ishim Replicate Runs

In [7]:
import numpy as np
import os as os
import sys as sys
import multiprocessing as mp
import matplotlib.pyplot as plt

import pandas as pd
import socket

### For Arial Font
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'   # Set the defaul
rcParams['font.sans-serif'] = ['Arial']  # Make sure to have the font installed (it is on cluster for Harald)

#######################################################
### Pick the right path (whether on cluster or at home)
socket_name = socket.gethostname()
print(socket_name)
if socket_name == "VioletQueen":
    path = "/home/harald/git/HAPSBURG/"   # The Path on Harald's machine
elif socket_name.startswith("midway2"):
    print("Midway jnovmbre partition detected.")
    path = "/project2/jnovembre/hringbauer/HAPSBURG/"  # The Path on Midway Cluster
    sys.path.append("/project2/jnovembre/hringbauer/HAPSBURG/package/")
else: 
    raise RuntimeWarning("Not compatible machine. Check!!")
    
os.chdir(path)  # Set the right Path (in line with Atom default)
print(os.getcwd()) # Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns
print(f"CPU Count: {mp.cpu_count()}")

### Import hapsburg Packages
from hapsburg.PackagesSupport.pp_individual_roh_csvs import create_combined_ROH_df, give_iid_paths, pp_individual_roh

midway2-0401.rcc.local
Midway jnovmbre partition detected.
/project2/jnovembre/hringbauer/HAPSBURG
CPU Count: 28


### Forumla to get the IDs over Replicates

In [8]:
def get_iid_rep(i, reps=70, down_sampling_covs = np.linspace(0.2, 1.0, 9),
                 base_path="./Data/SA_1240kHDF5/Ust_Ishim_rep/downsample_ph_r"):
    """Get the Individual IID"""
    batch = int(np.floor(i/reps))
    rep = i%reps
    c = f"{down_sampling_covs[batch]:.4f}"
    iid = f"{c}_r{rep}" 
    return iid, c

In [9]:
get_iid_rep(10)

('0.2000_r10', '0.2000')

### Combine Ust Ishim Replicates

In [11]:
n=100 * 9
iids = np.array([get_iid_rep(i, reps=100)[0] for i in range(n)])
clsts= np.array([get_iid_rep(i, reps=100)[1] for i in range(n)])
paths = give_iid_paths(iids, base_folder="./Empirical/1240k/SA_Readcounts/Ust_Ishim_rep/", suffix='_roh_full.csv')

In [12]:
%%time
df1 = create_combined_ROH_df(paths, iids, pops=clsts, min_cm=[4,8,12,20], snp_cm=50, 
                             gap=0.5, min_len1=2, min_len2=4,
                             output=False, sort=False)

CPU times: user 12min 33s, sys: 1.54 s, total: 12min 35s
Wall time: 12min 57s


In [13]:
### Save
path_save = "./Empirical/1240k/SA_Readcounts/Ust_Ishim_rep/combined_roh05.tsv"
df1.to_csv(path_save, sep="\t", index=False)
df1["pop"] = df1["pop"].astype("str")
print(f"Successfully saved {len(df1)} Individuals to {path_save}")

Successfully saved 900 Individuals to ./Empirical/1240k/SA_Readcounts/Ust_Ishim_rep/combined_roh05.tsv


### Combine IPK12 Replicates